#### Problem statement 

    Objective: 
    To get the top relevent customer reviews about the product so that company could get the insights about their products. 
    
    Assumptions:
    1. If I could show the top_n and botton_n reviews about the product based on the polarity(sentiment of review) and ratings, this could give insights to customers about their product best and worst features. 
    
    About Notebook:
    I started with EDA to get familier with data, then doing some data cleaning techniques followed with creating features using tfidf and countvectorizer. Used few Classification models like . Naivebayes, Randomforest and XGboost. 
    Choose Randomforest and predict the most relevent reviews. 
    
    Then I tried LSTM to further improve the F-score. 
    
    
    Future Enhancements:
    1. BERT base models could be implemented 
    2. More embedding techniques could be explored to make features. 

#### Importing libraries 

In [ ]:
import pandas as pd 
import seaborn as sns
pd.set_option('display.max_colwidth', None)
import contractions
import re 
from nltk.corpus import wordnet
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from textblob import TextBlob
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
import string
import spacy
nlp = spacy.load("en_core_web_sm")
import sklearn 
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from collections import Counter
from nltk.util import ngrams
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
tqdm.pandas()
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from textblob.sentiments import NaiveBayesAnalyzer
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
import pickle 

#### Getting datafram from csv file

In [ ]:
df = pd.read_csv(r'C:\LeadSQ\Womens Clothing E-Commerce Reviews.csv', index_col= 0)

#### Basic EDA 

In [ ]:
df.head(1)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
len(df[df.duplicated()])

In [ ]:
df.drop_duplicates(keep=False, inplace=True)

In [ ]:
df.info()

In [ ]:
df['Clothing ID'].value_counts() 
#1206 unique IDs 

In [ ]:
df[df['Clothing ID'] ==776].head(1)

In [ ]:
df['Rating'].value_counts()
# data is skewed

In [ ]:
df['Recommended IND'].value_counts()

In [ ]:
df['Positive Feedback Count'].value_counts()

In [ ]:
df[df['Positive Feedback Count'] == 59]

In [ ]:
ax = sns.countplot(data=df, x="Positive Feedback Count")
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
ax = sns.countplot(data=df, x="Rating")
for container in ax.containers:
    ax.bar_label(container)
    
# Data is highly skewed. 

In [ ]:
ax = sns.countplot(data=df, hue="Recommended IND",  x="Rating")
for container in ax.containers:
    ax.bar_label(container)
    
# we could see that even for rating = 5, there are 25 entries which has recommended ind = 0 
# we could see that even for rating = 1, there are 16 entries which has recommended ind = 1
# these all the outliers.. so
# for rating = 3, these are the ambiguous ratings. 

In [ ]:
df[(df['Rating'] == 5) & (df['Recommended IND'] ==0 )].head(1)

In [ ]:
df[(df['Rating'] == 1) & (df['Recommended IND'] ==1)].head(1)

In [ ]:
df[(df['Rating'] == 5) & (df['Recommended IND'] ==0 )].index

In [ ]:
df= df.drop(df[(df['Rating'] == 1) & (df['Recommended IND'] ==1)].index)
df= df.drop(df[(df['Rating'] == 2) & (df['Recommended IND'] ==1)].index)
df= df.drop(df[(df['Rating'] == 5) & (df['Recommended IND'] ==0 )].index)
df= df.drop(df[(df['Rating'] == 4) & (df['Recommended IND'] ==0 )].index)
df.reset_index(inplace=True)

In [ ]:
df.shape

In [ ]:
df['Division Name'].value_counts(dropna = False)

#manual rating (i will go data labelling team)
#remove the outliers 
#

In [ ]:
ax = sns.countplot(data=df, x="Division Name", hue="Recommended IND")
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
df['Department Name'].value_counts(dropna = False)

In [ ]:
ax = sns.countplot(data=df, x="Department Name", hue="Recommended IND")
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
df['Class Name'].value_counts(dropna = False)

In [ ]:
df[['Class Name','Clothing ID']].groupby(['Class Name']).value_counts()

In [ ]:
# check combined NULLs in Review text column and in Title columns.. These cud be combined to get full text. 
# doing this way wud reduce the NULL records. 
df_nulls = df[((df['Review Text'].isnull()) & df['Title'].isnull())]

In [ ]:
df_nulls.shape

In [ ]:
ax = sns.countplot(data=df_nulls,  x="Rating")
for container in ax.containers:
    ax.bar_label(container)
# total 844 such records - having null review_Text and title. 
# 590 such records which has null review text and Title... and still rating is 5. 

In [ ]:
df_nulls.head()

In [ ]:
df['Review Text']=df['Review Text'].apply(str)
df['Title']=df['Title'].apply(str)

# combining the Review Text and Title columns to get single columns .. Review 
df['Review'] = df['Title'] + ' ' + df['Review Text']
df['Review']=df['Review'].apply(str)
df['Review_len'] = df['Review'].str.len()

In [ ]:
df[['Review_len']].describe()

In [ ]:
df_train = df.iloc[:22000]
df_test = df.iloc[22000:]

#### Data Normalization 

In [ ]:
df_new = df_train[['Review', 'Recommended IND', 'Rating' ,'Review_len', 'Class Name']]

In [ ]:
def remove_repeated_characters(tokens):
    '''
    
    '''
    old_word = tokens
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')   #check this expression. 
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens


def process_text(document):
        document = re.sub(r'\s+', ' ', document, flags=re.I)  #to remove multiple white space character 
        document = re.sub(r'\W', ' ', str(document))
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        document = re.sub(r'[^a-zA-Z]', ' ', document)
        document = document.lower() 
        tokens = document.split()
        tokens = [contractions.fix(word) for word in tokens]
        tokens = remove_repeated_characters(tokens)
        #lemma_txt = [lemmatizer.lemmatize(word) for word in tokens]
        #lemma_no_stop_txt = [word for word in lemma_txt if word not in stopwords]
        clean_txt = ' '.join(tokens)
        return clean_txt
    

def wordcloud_text(data):
    text = " ".join(review for review in data)
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    
    
def get_pos(review):
    lis = []
    doc = nlp(review)
    for token in doc: 
        if (token.pos_ != 'PRON' and token.pos_ != 'DET' and token.pos_ != 'SCONJ' and token.pos_ != 'ADP'
        and token.pos_ != 'PROPN' and token.pos_ != 'CCONJ' and token.pos_ != 'AUX'):
            lis.append(token.text)
    return ' '.join(lis)


def get_most_common (reviews, top_n ):
    lis = []
    for reviews in reviews:
        for words in reviews.split():
            lis.append(words)
    my_counter = Counter(lis)
    top_n = my_counter.most_common(top_n)
    return(top_n)

def extract_ngrams(reviews, num):
    for review in reviews:
        n_grams = ngrams(nltk.word_tokenize(review), num)
        sent =  [ '_'.join(grams) for grams in n_grams]
    return (' '.join(sent))

def unique_words(data):
    lis = []
    for review in data:
        for word in review.split():
            lis.append(word)
    return (set(lis))



def model_train(model, X_train, y_train,X_test,y_test):
    model.fit(X_train, y_train)
    y_pred=model.predict(X_test)
    print(f"accuracy score for {model} is {accuracy_score(y_pred, y_test)}")
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    average_precision = average_precision_score(y_test, y_pred)
    print(f" average precision is {average_precision}")
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
    auc_precision_recall = auc(recall, precision)
    print(f"auc precison recall is {auc_precision_recall}")
    plt.plot(recall, precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    plt.show()
    return classification_report(y_test, y_pred)


def remov_letter (review, n_char):
    lis = []
    for word in review.split(): 
        if (len(word)>n_char):
            lis.append(word)
    return ' '.join(lis)


def lemmatiz(document):
        tokens = document.split()
        lemma_txt = [lemmatizer.lemmatize(word) for word in tokens]        
        clean_txt = ' '.join(lemma_txt)
        return clean_txt
    
    
def get_rating (row):
    if (row['prdictions'] == 1) & (row['polarity']> df_polarity[df_polarity['Rating'].isin([4,5])]['polarity_mean'].mean()):
        return 5 
    if (row['prdictions'] == 1) & (row['polarity']<= df_polarity[df_polarity['Rating'].isin([4,5])]['polarity_mean'].mean()):
        return 4
    if (row['prdictions'] == 0) & (row['polarity']>= df_polarity[df_polarity['Rating'].isin([2,3])]['polarity_mean'].mean()):
        return 3
    if (row['prdictions'] == 0) & (row['polarity']>= df_polarity[df_polarity['Rating'].isin([1,2])]['polarity_mean'].mean()):
        return 2
    if (row['prdictions'] == 0) & (row['polarity']<= df_polarity[df_polarity['Rating'].isin([2])]['polarity_mean'].mean()):
        return 1
    

df_polarity=df_new.groupby('Rating').agg({'polarity':'mean'}).rename(columns={'polarity':'polarity_mean'}).reset_index()

    

In [ ]:
df_new['processed_review'] = df_new['Review'].progress_apply(process_text)
df_new['processed_review_len'] = df_new['processed_review'].str.len()
df_new[['processed_review_len', 'Review_len']].describe()

In [ ]:
review = ' nan was really hoping to like this but it did not look the way it does on the model at least not on me the sharkbite hem is much more pronounced and looser the one in the photo looks like it was pinned back am and usually wear medium or large got medium and there was lot more material on the bottom half than the photo shows it made me look bigger and was not flattering material has nice weave but it thin and delicate bought the holly deep olive and the blue colors'
doc = nlp(review)
for token in doc: 
    if (token.pos_ != 'PRON' and token.pos_ != 'DET' and token.pos_ != 'SCONJ' and token.pos_ != 'ADP'
        and token.pos_ != 'PROPN' and token.pos_ != 'CCONJ' and token.pos_ != 'AUX' and token.pos_ != 'NOUN'):
        print(token.pos_, token.text)

In [ ]:
wordcloud_text(df_new[df_new.Rating <3].processed_review)
# to know about the high level what are the most occuring words for rating <3 

In [ ]:
wordcloud_text(df_new[df_new.Rating >3].processed_review)

In [ ]:
df_new['only_adj_Adv'] = df_new.processed_review.progress_apply(lambda x :get_pos(x))
df_new['only_adj_Adv_len'] = df_new['only_adj_Adv'].str.len()

In [ ]:
wordcloud_text(df_new[df_new.Rating >3].only_adj_Adv)
wordcloud_text(df_new[df_new.Rating <3].only_adj_Adv)

In [ ]:
df_new['polarity'] = df_new['only_adj_Adv'].progress_apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
df_new['subjectivity'] = df_new['only_adj_Adv'].progress_apply(lambda x: TextBlob(x).sentiment[1])

In [ ]:
df_grp1 = df_new.groupby('Rating').agg({'subjectivity':'mean', 'polarity':'mean'}).rename(columns={'polarity':'polarity_mean', 'subjectivity':'subj_mean'}).reset_index()
df_grp1

In [ ]:
df_grp = df_new.groupby('Recommended IND').agg({'polarity':'mean', 'only_adj_Adv_len':'mean'}).rename(columns={'polarity':'polarity_mean', 'only_adj_Adv_len':'sent_len_mean'}).reset_index()
df_grp

In [ ]:
freq_word1 = get_most_common(df_new[df_new.Rating > 3].only_adj_Adv, 200)
freq_word2 = get_most_common(df_new[df_new.Rating < 3].only_adj_Adv, 200)
print(freq_word1)
print(freq_word2)

In [ ]:
print(len(unique_words(df_new[df_new.Rating < 3].only_adj_Adv)))
print(len(unique_words(df_new[df_new.Rating == 3].only_adj_Adv)))
print(len(unique_words(df_new[df_new.Rating > 3].only_adj_Adv)))
len(unique_words(df_new.only_adj_Adv))

In [ ]:
print(df_new[df_new['only_adj_Adv_len']==0].shape)
df_new = df_new.drop(df_new[df_new['only_adj_Adv_len']==0].index)
df_new.reset_index(inplace=True)

In [ ]:
df_new[['processed_review_len', 'Review_len', 'only_adj_Adv_len']].describe()

#### using countvectorizer

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(df_new['only_adj_Adv'], df_new['Recommended IND'],stratify= df_new['Recommended IND'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df = .0001, max_df = .99, ngram_range = (1,1))

In [ ]:
cv_xtrain = cv.fit_transform(Xtrain)
cv_xtest = cv.transform(Xtest)
#cv.vocabulary_

In [ ]:
count_matrix = cv_xtrain.toarray()
count_matrix.shape

#### using tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df = .0002, max_df = .99, ngram_range = (1,1))

In [ ]:
tfidf_Xtrain = tfidf.fit_transform(Xtrain)
tfidf_Xtest = tfidf.transform(Xtest)
#cv.vocabulary_

In [ ]:
tfidf_matrix = tfidf_Xtrain.toarray()
tfidf_matrix.shape

#### Selecting model 

In [ ]:
output = model_train(MultinomialNB(), tfidf_Xtrain, Ytrain, tfidf_Xtest ,Ytest )
print(output)

In [ ]:
output = model_train(MultinomialNB(), cv_xtrain, Ytrain, cv_xtest ,Ytest )
print(output)

In [ ]:
output = model_train(RandomForestClassifier(n_estimators=1000, max_depth=5, 
        class_weight='balanced', random_state=3), cv_xtrain, Ytrain, cv_xtest ,Ytest)
print(output)

In [ ]:
output = model_train(RandomForestClassifier(n_estimators=1000, max_depth=5, 
        class_weight='balanced', random_state=3), tfidf_Xtrain, Ytrain, tfidf_Xtest ,Ytest)
print(output)

In [ ]:
output = model_train(XGBClassifier(), cv_xtrain, Ytrain, cv_xtest ,Ytest)
print(output)

#### Creating model, Dumping and Loading 

In [ ]:
pickle.dump(tfidf, open("tfidfvectorizer.pickle", "wb"))
Rfmodel = RandomForestClassifier(n_estimators=1000, max_depth=5, class_weight='balanced', random_state=3)
Rfmodel.fit(tfidf_Xtrain, Ytrain)
pickle.dump(Rfmodel, open("rf.pickle", "wb"))

In [ ]:
vectorizer = pickle.load(open("tfidfvectorizer.pickle", "rb"))
rf = pickle.load(open("rf.pickle", "rb"))

#### Prediction 

In [ ]:
df_test['Clothing ID'].value_counts()

In [ ]:
df_test[df_test['Clothing ID'] == 1094]['Class Name'].value_counts()

In [ ]:
def predict(clothids,className,Topn):
    df1 = pd.DataFrame()
    df_pred = pd.DataFrame()
    df_pred = df_test[(df_test['Clothing ID'].isin(clothids)) & (df_test['Class Name'].isin(className))]
    if (df_pred.shape[0] >0):
        print(df_pred.shape[0])
        df_pred['Review'] = df_pred['Title'] + ' ' + df_pred['Review Text']
        df_pred['Review']=df_pred['Review'].apply(str)
        df_pred['processed_review'] = df_pred['Review'].progress_apply(process_text)
        df_pred['only_adj_Adv'] = df_pred.processed_review.progress_apply(lambda x :get_pos(x))
        df_pred['polarity'] = df_pred['only_adj_Adv'].apply(lambda x: TextBlob(x).sentiment.polarity)
        df_pred['only_adj_Adv_len'] = df_pred['only_adj_Adv'].str.len()
        df_pred = df_pred.drop(df_pred[df_pred['only_adj_Adv_len']==0].index)
        df_pred.reset_index(inplace=True)
        print(df_pred.shape)
        vectorizer = pickle.load(open("tfidfvectorizer.pickle", "rb"))
        tfidf_pred = vectorizer.transform(df_pred['only_adj_Adv'])
        df_pred['prdictions'] = rf.predict(tfidf_pred).tolist()
        df_pred['Predicted_Ratings'] = df_pred.progress_apply(lambda row:get_rating(row), axis=1)
        df_fetch = df_pred[['Clothing ID', 'prdictions', 'Predicted_Ratings', 'polarity', 'Review Text', 'Title', 'Class Name']]
        for id in clothids:
            print(id)
            for clas in className:
                print(clas)
                df_pos= df_fetch[(df_fetch['Clothing ID'] == id) & (df_fetch['Class Name'] == clas)].sort_values(by=['Predicted_Ratings','polarity'], ascending=False)
                df1 = df1.append(df_pos[:Topn], ignore_index = True)
                df_pos.drop(df_pos.index, inplace=True)
                df_neg= df_fetch[(df_fetch['Clothing ID'] == id) & (df_fetch['Class Name'] == clas)].sort_values(by=[ 'Predicted_Ratings','polarity'], ascending=True)
                df1 = df1.append(df_neg[:Topn], ignore_index = True)
                df_neg.drop(df_neg.index, inplace=True)
    return df1
    

In [ ]:
predict([1094, 1087],['Knits', 'Dresses'],3)

#### Using LSTM

In [ ]:
df['Review Text']=df['Review Text'].apply(str)

In [ ]:
Xtrain_lstm, Xtest_lstm, Ytrain_lstm, Ytest_lstm = train_test_split(df['Review Text'], df['Recommended IND'],stratify= df['Recommended IND'], 
                                                                   random_state=42)

In [ ]:
tokenizer = Tokenizer(oov_token='UNK')
tokenizer.fit_on_texts(Xtrain_lstm)

vocab_size = len(tokenizer.word_index)
print('Vocabulary size:', vocab_size)

In [ ]:
X_train_sequences = tokenizer.texts_to_sequences(Xtrain_lstm)
X_train_padded_seq = pad_sequences(X_train_sequences, maxlen=300, padding='post', truncating='post')
print(X_train_padded_seq.shape)

In [ ]:
max_features = 13161 
maxlen =  300 

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))(x)
x = layers.Bidirectional(layers.LSTM(64,dropout=0.5))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(X_train_padded_seq, Ytrain_lstm, batch_size=32, epochs=1, verbose=1)
model.save('lstm_model.h5')

In [ ]:
pickle.dump(tokenizer, open("lstmtokenizer.pickle", "wb"))

In [ ]:
from keras.models import load_model

In [ ]:
model1 = load_model('lstm_model.h5')
lstmvectorizer = pickle.load(open("lstmtokenizer.pickle", "rb"))

In [ ]:
X_test_sequences = lstmvectorizer.texts_to_sequences(Xtest_lstm)
X_test_padded_seq = pad_sequences(X_test_sequences, maxlen=300, padding='post', truncating='post')
predictions = model1.predict(X_test_padded_seq)

In [ ]:
test_pred = pd.DataFrame(predictions)  
test_pred = test_pred.reset_index()

In [ ]:
test_df = pd.concat([Xtest_lstm,Ytest_lstm], axis=1 )  
test_df = test_df.reset_index()

In [ ]:
test_pred.shape

In [ ]:
test_df.shape

In [ ]:
def fun1(x):
    if x >=0.5:
        x= 1 
    else:
        x = 0
    return x

In [ ]:
horizontal_Stack = pd.merge(test_pred, test_df, left_index=True, right_index=True)
horizontal_Stack.drop(['index_x','index_y' ],  axis=1, inplace=True)
horizontal_Stack['y_pred'] = horizontal_Stack[0].map(lambda x:fun1(x))

In [ ]:
horizontal_Stack .shape

In [ ]:
print(classification_report(horizontal_Stack['Recommended IND'], horizontal_Stack['y_pred']))